In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant U data

### Reading the files and extracting relevant information:

In [5]:
plant = "U"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  U ['INSTRUÇÕES', 'Clínquer', '103']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('U', '103', array(['CPIIE32'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIIE32': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [13]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [14]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    # .str.replace("D-G", "")
    # .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [15]:
NUMERIC_COLUMNS = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [16]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [17]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
CS1,100.000000
TiO2,100.000000
Specific Gravity,100.000000
#400,100.000000
Insoluble Residue,6.666667
Loss on Ignition,4.000000
CS28,3.288889
Blaine,3.022222
Final setting time,2.666667
Initial setting time,2.666667


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant U - Full Dataset:</th>
    <th></th>
    <tr>
        <td>CS1</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>#400</td>
        <td>100.00</td>
    </tr>      
    <tr>
        <td>TiO2</td>
        <td>100.00</td>
    </tr>        
</table>

In [18]:
FEATRUES_TO_DROP = ["CS1", "TiO2", "Specific Gravity", "#400"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with many zeros when applicable

In [19]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for U
Na2O,0.622222
CaO,0.000000
Blaine,0.000000
CS7,0.000000
CS3,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#325,0.000000
#200,0.000000
Insoluble Residue,0.000000


<IPython.core.display.Javascript object>

### Dropping variable with many zeros

In [20]:
# FEATRUES_TO_DROP = ["#200"]
# df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [21]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [22]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [23]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [24]:
df.shape

(1088, 20)

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [25]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [26]:
df.shape

(1081, 20)

<IPython.core.display.Javascript object>

In [27]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [28]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [29]:
df.shape

(1076, 20)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [30]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

CaO                     0.0
MgO                     0.0
CS7                     0.0
CS3                     0.0
Final setting time      0.0
Initial setting time    0.0
#325                    0.0
#200                    0.0
Blaine                  0.0
Insoluble Residue       0.0
Loss on Ignition        0.0
Fe2O3                   0.0
K2O                     0.0
SO3                     0.0
SiO2                    0.0
Al2O3                   0.0
Na2O                    0.0
CS28                    0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [31]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,1076,2022-04-24 07:04:14.275092992,2021-01-04 00:00:00,2022-05-14 12:00:00,2023-07-10 06:00:00,2023-07-19 00:00:00,NaN
CaO,1066.0,56.03363,51.330002,56.029999,59.548,62.919998,1.349958
MgO,1066.0,2.999841,0.07,2.99,3.937,4.14,0.419806
Na2O,1066.0,0.10743,0.01,0.09,0.12,9.0,0.358957
Al2O3,1066.0,5.894568,2.22,5.77,6.3335,58.599998,2.670755
SiO2,1066.0,23.468902,2.33,21.49,23.227499,2125.0,64.435116
SO3,1065.0,2.31108,0.89,2.31,2.73,2.97,0.154705
K2O,1066.0,0.966792,0.07,0.88,0.99,86.0,2.608236
Fe2O3,1066.0,2.844536,1.89,2.515,3.31,258.0,8.040596
Loss on Ignition,1038.0,6.8179,1.74,6.87,8.5,8.67,0.852061


<IPython.core.display.Javascript object>

In [32]:
df = df.reset_index(drop=True)

df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)
df = df[~df["Na2O"].eq(df["Na2O"].max())].reset_index(drop=True)

df = df[~df["Al2O3"].eq(df["Al2O3"].max())].reset_index(drop=True)
df = df[~df["Al2O3"].eq(df["Al2O3"].max())].reset_index(drop=True)
df = df[~df["Al2O3"].eq(df["Al2O3"].max())].reset_index(drop=True)
df = df[~df["Al2O3"].eq(df["Al2O3"].max())].reset_index(drop=True)

df = df[~df["SiO2"].eq(df["SiO2"].max())].reset_index(drop=True)
df = df[~df["K2O"].eq(df["K2O"].max())].reset_index(drop=True)

df = df[~df["Fe2O3"].eq(df["Fe2O3"].max())].reset_index(drop=True)
df = df[~df["Fe2O3"].eq(df["Fe2O3"].max())].reset_index(drop=True)
df = df[~df["Fe2O3"].eq(df["Fe2O3"].max())].reset_index(drop=True)


df = df[~df["Final setting time"].eq(df["Final setting time"].max())].reset_index(
    drop=True
)
df = df[~df["Final setting time"].eq(df["Final setting time"].min())].reset_index(
    drop=True
)
df = df[~df["Final setting time"].eq(df["Final setting time"].min())].reset_index(
    drop=True
)


<IPython.core.display.Javascript object>

In [33]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,1056,2022-04-25 11:08:10.909091072,2021-01-04 00:00:00,2022-05-16 12:00:00,2023-07-10 10:48:00,2023-07-19 00:00:00,NaN
CaO,1046.0,56.035908,51.700001,56.029999,59.4795,62.919998,1.34132
MgO,1046.0,3.000201,0.07,2.99,3.941,4.14,0.418029
Na2O,1046.0,0.087151,0.01,0.09,0.12,0.22,0.019998
Al2O3,1046.0,5.744579,2.22,5.77,6.301,6.57,0.327678
SiO2,1046.0,21.495822,2.33,21.5,23.2055,34.5,1.056205
SO3,1045.0,2.311368,0.89,2.31,2.73,2.97,0.155274
K2O,1046.0,0.887294,0.07,0.88,0.99,2.88,0.085321
Fe2O3,1046.0,2.544919,1.89,2.51,3.2355,5.77,0.291253
Loss on Ignition,1018.0,6.817544,1.74,6.87,8.5,8.67,0.851593


<IPython.core.display.Javascript object>

In [34]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Insoluble Residue,6.344697
Loss on Ignition,3.598485
Blaine,2.556818
Initial setting time,2.272727
Final setting time,2.272727
CS3,1.136364
SO3,1.041667
CS7,1.041667
MgO,0.946970
Na2O,0.946970


<IPython.core.display.Javascript object>

In [35]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1056,2022-04-25 11:08:10.909091072,2021-01-04 00:00:00,2021-09-03 18:00:00,2022-05-16 12:00:00,2022-12-02 00:00:00,2023-07-19 00:00:00,NaN
CaO,1046.0,56.035908,51.700001,55.2025,56.029999,56.8875,62.919998,1.34132
MgO,1046.0,3.000201,0.07,2.7,2.99,3.33,4.14,0.418029
Na2O,1046.0,0.087151,0.01,0.08,0.09,0.1,0.22,0.019998
Al2O3,1046.0,5.744579,2.22,5.58,5.77,5.93,6.57,0.327678
SiO2,1046.0,21.495822,2.33,20.892499,21.5,22.09,34.5,1.056205
SO3,1045.0,2.311368,0.89,2.22,2.31,2.4,2.97,0.155274
K2O,1046.0,0.887294,0.07,0.85,0.88,0.92,2.88,0.085321
Fe2O3,1046.0,2.544919,1.89,2.39,2.51,2.6475,5.77,0.291253
Loss on Ignition,1018.0,6.817544,1.74,6.18,6.87,7.43,8.67,0.851593


<IPython.core.display.Javascript object>

In [36]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1056,NaN,NaN,NaN,2022-04-25 11:08:10.909091072,2021-01-04 00:00:00,2021-09-03 18:00:00,2022-05-16 12:00:00,2022-12-02 00:00:00,2023-07-19 00:00:00
Cement_Type,1056,2,CP II-E-32 D-G,549,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1056 entries, 0 to 1055
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1056 non-null   datetime64[ns]
 1   CaO                   1046 non-null   float64       
 2   MgO                   1046 non-null   float64       
 3   Na2O                  1046 non-null   float64       
 4   Al2O3                 1046 non-null   float64       
 5   SiO2                  1046 non-null   float64       
 6   SO3                   1045 non-null   float64       
 7   K2O                   1046 non-null   float64       
 8   Fe2O3                 1046 non-null   float64       
 9   Loss on Ignition      1018 non-null   float64       
 10  Insoluble Residue     989 non-null    float64       
 11  Blaine                1029 non-null   float64       
 12  #200                  1055 non-null   float64       
 13  #325              

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [38]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [39]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [40]:
df.to_csv("../../../../data/interim/209/u.csv", index=False)

<IPython.core.display.Javascript object>